In [ ]:
import pandas as pd
import numpy as np

In [ ]:
queries = []
with open("data/queries.txt") as f:
    queries = f.read().splitlines()

In [ ]:
movie_info = pd.read_csv("data/movies.csv")

In [ ]:
def append_info(movie_info, new_movie_name):
    cnt = 0
    movie_ids = []
    original_movie_names = []
    movie_descs = []
    for movie_name, movie_desc in new_movie_name:
        filter_condition = movie_info["title"].apply(lambda x: x.lower().split("(")[0].strip() == movie_name.lower())
        # filter_condition = movie_info["title"].apply(lambda x: x.lower().startswith(movie_name.lower()))
        movie_id = movie_info.loc[filter_condition]
        if len(movie_id) != 0:
            movie_id = movie_id.iloc[-1]["movieId"]
            original_movie_name = movie_info.loc[movie_info["movieId"] == movie_id]["title"].values[0]
            print(movie_id, movie_name, original_movie_name, movie_desc)
            movie_ids.append(movie_id)
            original_movie_names.append(original_movie_name)
            movie_descs.append(movie_desc)
            cnt += 1
    print("Total: ", cnt)
    new_df = pd.DataFrame({"id": movie_ids, "title": original_movie_names, "desc": movie_descs})
    return new_df

Start with query

In [ ]:
qid = 48
query = queries[qid]
print("Can you give me 20 relevant movies for query: ", query, "?")
# print("Please come up with 20 negative but confusing samples of the query: ", query, "?")


ChatGPT answers

In [ ]:
with open("new_movie.txt") as f:
    new_movie = f.read().splitlines()

def fetch_movie_name(movie_desc):
    return movie_desc.split('.')[1].split(" (")[0].strip(), movie_desc.split("-")[1].strip()

new_movie_name = list(map(fetch_movie_name, new_movie))
new_df = append_info(movie_info, new_movie_name)


In [ ]:
new_movie_info = {k: v for k, v in zip(new_df["title"], new_df["desc"])}

In [ ]:
query_file = pd.read_csv("data/query{}.csv".format(qid+1))
query_file = pd.concat([new_df, query_file])
query_file["rel"] = 1
movies = query_file["title"].values

Parameters:
- qid
- mode
  - "a": output for gpt
  - other: input rel

In [ ]:
mode = "q"

In [ ]:
mode = "a"

In [ ]:
print("Query: ", query)
for i, movie in enumerate(movies):
    if mode != "a":
        score = input("Movie: {}, {} ".format(movie, new_movie_info.get(movie, "")))
        # score = new_movie_info.get(movie, "")
        try:
            if score == "":
                score = 1
            else:
                score = int(score)
            print("Movie: ", movie, "relevance score will be: ", score)
        except:
            break
        query_file.loc[query_file["title"] == movie, "rel"] = score
    else:
        print(f"{i}. {movie}")

In [ ]:
query_file.sort_values(by="rel", ascending=False, inplace=True)

In [ ]:
# drop the second duplicate
query_file = query_file.drop_duplicates(subset=["title"], keep="first")
query_file

In [ ]:
query_file.drop(columns='desc', inplace=True)

In [ ]:
query_file[:60]

In [ ]:
with open("annotation/query{}.csv".format(qid+1), "w") as f:
    query_file[:60].to_csv(f, index=False)

In [ ]:
qid

In [ ]:
query_file['rel'].value_counts()

In [ ]:
qid = 48
print("Can you give me 20 relevant movies for query: ", queries[qid], "?")
a = pd.read_csv("annotation/query{}.csv".format(qid+1))
a.drop_duplicates(subset=["title"], keep="first", inplace=True)
a.sort_values(by="rel", ascending=False, inplace=True)
a.drop(columns='desc', inplace=True)
a
a.to_csv("annotation/query{}.csv".format(qid+1), index=False)